### Imports

**REFER TO DATA_COLLECTION NOTEBOOK IF YOU WANT TO SEE REQUESTS METHOD FOR SCRAPING, THIS NOTEBOOK USES PRAW**

In [2]:
# !pip install praw

In [3]:
import pandas as pd 
import numpy as np 
import time 
import praw

### API credentials 

In [4]:
reddit = praw.Reddit(
    client_id = 'NSeSVr03ZJ8ov2YBVvK6vw',
    client_secret =  'VJbMqoBH0yMzhUXtvrtBUY6IgwLcXQ',
    user_agent = 'navi dsb-826',
    username = 'OkCommunity6752',
    password =  'Magnum12' 
)

In [5]:
# Read the existing CSV filed 
def load_existing_data(file):
    try:
        return pd.read_csv(file)
    except FileNotFoundError: 
        # return an empty dataframe if file does not exist
        return pd.DataFrame(columns=['subreddit', 'title', 'body', 'created', 'score', 'num_comments', 'id'])

In [6]:
# Create function to get posts from subreddit
def get_old_posts(subreddit_name, limit=100):
    subreddit = reddit.subreddit(subreddit_name)

    # Create list of posts to append to 
    posts_list = []

    # Get posts 
    for post in subreddit.new(limit = limit): 
        posts_list.append({
            'subreddit': subreddit_name, 
            'title': post.title,
            'body': post.selftext,
            'created': post.created_utc,
            'score': post.score,
            'num_comments': post.num_comments,
            'id': post.id
        })
    return pd.DataFrame(posts_list)

In [7]:
# Function to filter unique posts
def unique_filter(df, new_posts):
    return new_posts[~new_posts['id'].isin(df['id'])]

### Function to Pull data 

I initially used requests to pull data from Reddit's API. However, after attending office hours, I learned about praw and decided to try it out myself. Using praw helped simply the processing of getting data on a day to day basis as I simply had to run one block of code to pull data. If you want to see how I used requests to scrape my data, refer to the other data collection notebook labelled "data-collection." 

In [8]:
def update_posts(subreddit_name, file, limit = 100):
    print(f'Collecting posts from {subreddit_name}')

    # Load the existing posts 
    existing_posts = load_existing_data(file)

    # Get new posts 
    new_posts = get_old_posts(subreddit_name, limit = limit)

    # Filter out duplicates
    unique_posts = unique_filter(existing_posts, new_posts)

    if not unique_posts.empty: 
        # Append unique posts to exisiting csv 
        all_posts = pd.concat([existing_posts, unique_posts], ignore_index = True)
        all_posts.drop_duplicates(subset = ['id'], inplace = True)

        # Save the data
        all_posts.to_csv(file, index = False)
        print(f'{len(unique_posts)} new posts added from {subreddit_name}. Total posts: {len(all_posts)}')
    else: 
        print(f'No new posts for {subreddit_name}')

### Collect Data 

In [10]:
subreddits = ['personalfinance', 'investing']
csv_file = '../data/subreddit-data.csv'

for subreddit in subreddits: 
    update_posts(subreddit, csv_file, limit = 100)

80 new posts added from personalfinance. Total posts: 2628
15 new posts added from investing. Total posts: 2643
